In [11]:
import sys
 
# adding Folder_2 to the system path
sys.path.insert(0, '../utils')

from typing import Dict
import argparse

from config import Config
from pgConnect import PgConnection
from dfs_dao import Dfs_dao
from requestLimiter import RequestLimiter
from teamRosterReader import TeamRosterReader, learn_teams_from_summary
from bs4 import BeautifulSoup
from bs4.element import Tag
import pandas as pd

from bs4utils import get_ith_table

In [12]:
config : Config = Config()
pgc : PgConnection = PgConnection(config)

# reader 
read_constants : Dict[str, str] = config.parse_section('reader')
BASE : str = read_constants['base']
NAME : str = BASE[BASE.find('.') + 1:]

# requestLimiter
rl_constants : Dict[str, str] = config.parse_section('requestLimiter')
load_loc = rl_constants['load_location']
LOAD_FILE : str = f'{load_loc}{NAME}.p'
INTERVAL : int = int(rl_constants['interval'])
LIMIT : int = int(rl_constants['limit'])

# # ======
# # 2. Parse args
# # ======
# parser = argparse.ArgumentParser()
# parser.add_argument("year", help = "Year of data to be added ")
# args = parser.parse_args()
# YEAR : int = int(args.year)

rl : RequestLimiter = RequestLimiter(BASE, 
                    interval = INTERVAL, 
                    limit = LIMIT - 1, 
                    load = LOAD_FILE)

Connecting to the PostgreSQL database...
In constructor...
Successfully loaded previous Rate Limiter info for https://www.basketball-reference.com
Initialized with 0 of 19 entries filled

Saving RequestLimiter status to disk...


In [13]:
link = 'https://www.basketball-reference.com/draft/NBA_2022.html'
res = rl.get(link, waitForPop = True)

Successfully processed append to queue...
Size of current queue... 1
Saving RequestLimiter status to disk...


In [17]:
data = res.text
soup : BeautifulSoup = BeautifulSoup(data, 'html.parser')
soup
table : Tag = get_ith_table(soup, 0, id = 'stats')

In [40]:
thead = table.find('thead')
data = []
column_names = [th.text.strip() for th in thead.find_all('th')]
column_names = column_names[column_names.index('Advanced') + 1:]
had_link = set()

for row in table.find_all(['tr']):
    row_data = []
    link_data = []
    for num, td in enumerate(row.find_all(['td','th'])):
        not_link = None
        if ''.join(td.text.strip()):
            not_link = ''.join(td.text.strip())
        row_data.append(not_link)

        if td.find('a'):
            link = td.a['href']
            link_data.append(link)
            had_link.add(num)
    data.append(row_data + link_data)
data
had_link = list(had_link)
had_link.sort()
for val in had_link:
    column_names.append(column_names[val] + '_link')

df = pd.DataFrame(data[2:], columns= column_names)
my_res = df[['Pk', 'Tm', 'Player', 'College', 'Player_link']]
# df

In [44]:
rows = []
for _, row in my_res.iterrows():
    if row['Pk'] is not None:
        out = (row['Pk'],
                row['Tm'],
                row['Player'],
                row['College'],
                row['Player_link'])
        rows.append(out)
rows

[('1', 'ORL', 'Paolo Banchero', 'Duke', '/players/b/banchpa01.html'),
 ('2', 'OKC', 'Chet Holmgren', 'Gonzaga', '/players/h/holmgch01.html'),
 ('3', 'HOU', 'Jabari Smith Jr.', 'Auburn', '/players/s/smithja05.html'),
 ('4', 'SAC', 'Keegan Murray', 'Iowa', '/players/m/murrake02.html'),
 ('5', 'DET', 'Jaden Ivey', 'Purdue', '/players/i/iveyja01.html'),
 ('6', 'IND', 'Bennedict Mathurin', 'Arizona', '/players/m/mathube01.html'),
 ('7', 'POR', 'Shaedon Sharpe', 'Kentucky', '/players/s/sharpsh01.html'),
 ('8', 'NOP', 'Dyson Daniels', None, '/players/d/daniedy01.html'),
 ('9', 'SAS', 'Jeremy Sochan', 'Baylor', '/players/s/sochaje01.html'),
 ('10', 'WAS', 'Johnny Davis', 'Wisconsin', '/players/d/davisjo06.html'),
 ('11', 'NYK', 'Ousmane Dieng', None, '/players/d/diengou01.html'),
 ('12', 'OKC', 'Jalen Williams', 'Santa Clara', '/players/w/willija06.html'),
 ('13', 'CHO', 'Jalen Duren', 'Memphis', '/players/d/durenja01.html'),
 ('14', 'CLE', 'Ochai Agbaji', 'Kansas', '/players/a/agbajoc01.html'

In [ ]:
def process_player_table(self, table : Tag) -> pd.DataFrame:
        """
        Takes HTML table and adds links before creating pd.DataFrame
        """
        table = table
        thead = table.find('thead')
        data = []
        column_names = [th.text.strip() for th in thead.find_all('th')]
        had_link = set()
        for row in table.find_all(['tr']):
            row_data = []
            link_data = []
            for num, td in enumerate(row.find_all(['td','th'])):
                not_link = None
                if ''.join(td.text.strip()):
                    not_link = ''.join(td.text.strip())
                row_data.append(not_link)

                if td.find('a'):
                    link = td.a['href']
                    link_data.append(link)
                    had_link.add(num)
            data.append(row_data + link_data)

        had_link = list(had_link)
        had_link.sort()
        for val in had_link:
            column_names.append(column_names[val] + '_link')

        df = pd.DataFrame(data[1:], columns= column_names)
        return df